In [ ]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# path where resulting .csv files for cases and stringencies are stored

csv_filepath = os.path.abspath(os.path.join(os.pardir, os.pardir, 'prescriptions'))
csv_filepath

In [ ]:
# dictionary storing prescriptor files in format model_name: path_to_file
prescription_files = {
    'BlindGreedy': os.path.join(csv_filepath, 'blind_greedy_evaluate.csv'),
    'Neat': os.path.join(csv_filepath, 'neat_evaluate.csv')
}

In [ ]:
def load_data(filepath):
    latest_df = pd.read_csv(filepath,
                            encoding="ISO-8859-1",
                            dtype={"RegionName": str},
                            error_bad_lines=False)
    return latest_df

In [ ]:
# Collect case and stringency data for all prescriptors
dfs = []
for prescriptor_name, prescription_file in sorted(prescription_files.items()):    
    df = load_data(prescription_file)
    df['PrescriptorName'] = prescriptor_name
    dfs.append(df)
df = pd.concat(dfs)

In [ ]:
# Compute domination relationship for each pair of prescriptors for each geo
def compute_domination_df(df, verbose=False):
    country_names = []
    region_names = []
    dominating_names = []
    dominated_names = []
    dominated_idxs = []
    prescriptor_names = sorted(df['PrescriptorName'].unique())  # Greedy, Greedy2
    prescription_idxs = sorted(df['PrescriptionIndex'].unique())  # 0-9
    for country_name in df['CountryName'].unique():
        cdf = df[df['CountryName'] == country_name]
        for region_name in cdf['RegionName'].unique():
            if verbose:
                print('Computing domination for', country_name, region_name)
            if pd.isna(region_name):
                rdf = cdf[cdf['RegionName'].isna()]
            else:
                rdf = cdf[cdf['RegionName'] == region_name]
            for name1 in prescriptor_names:
                dominated_prescriptions = set() # To collect all dominated prescriptions
                                                # this is a set, so each dominated
                                                # prescription is only counted once per
                                                # submission.
                name1_df = rdf[rdf['PrescriptorName'] == name1]
                name1_idxs = sorted(name1_df['PrescriptionIndex'].unique())
                for i in name1_idxs:
                    idf = name1_df[name1_df['PrescriptionIndex'] == i]
                    i_cases = idf['PredictedDailyNewCases'].values[0]
                    i_stringency = idf['Stringency'].values[0]
                    
                    # Compare name1 to all other prescriptions
                    for name2 in prescriptor_names:
                        if name1 != name2:
                            name2_df = rdf[rdf['PrescriptorName'] == name2]
                            name2_idxs = sorted(name2_df['PrescriptionIndex'].unique())  
                            for j in name2_idxs:
                                jdf = name2_df[name2_df['PrescriptionIndex'] == j]
                                j_cases = jdf['PredictedDailyNewCases'].values[0]
                                j_stringency = jdf['Stringency'].values[0]
                                if (i_cases < j_cases) and (i_stringency < j_stringency):
                                    dominated_prescriptions.add((name2, j))
                for name2, j in dominated_prescriptions:
                    country_names.append(country_name)
                    region_names.append(region_name)
                    dominating_names.append(name1)
                    dominated_names.append(name2)
                    dominated_idxs.append(j)
    ddf = pd.DataFrame({
        'CountryName': country_names,
        'RegionName': region_names,
        'DominatingName': dominating_names,
        'DominatedName': dominated_names,
        'DominatedIndex': dominated_idxs})
    return ddf

In [ ]:
# Get number of dominated prescriptions for each submission on each test. This is the "Domination Count"
for test_name in df['TestName'].unique():
    print('Test', test_name)
    df_test = df[df['TestName'] == test_name]
    ddf = compute_domination_df(df_test)
    print(ddf.groupby('DominatingName').count().sort_values('DominatedIndex', ascending=False)['DominatedIndex'])
    print()

In [ ]:
#
# Helpful functions for visualizing the area dominated by a set of solutions.
#

def compute_pareto_set(objective1_list, objective2_list):
    """
    Return objective values for the subset of solutions that
    lie on the pareto front.
    """
    
    assert len(objective1_list) == len(objective2_list), \
            "Each solution must have a value for each objective."
        
    n_solutions = len(objective1_list)
    
    objective1_pareto = []
    objective2_pareto = []
    for i in range(n_solutions):
        is_in_pareto_set = True
        for j in range(n_solutions):
            if (objective1_list[j] < objective1_list[i]) and \
                    (objective2_list[j] < objective2_list[i]):
                is_in_pareto_set = False
        if is_in_pareto_set:
            objective1_pareto.append(objective1_list[i])         
            objective2_pareto.append(objective2_list[i])   
    
    return objective1_pareto, objective2_pareto

    
def plot_pareto_curve(objective1_list, objective2_list):
    """
    Plot the pareto curve given the objective values for a set of solutions.
    This curve indicates the area dominated by the solution set, i.e., 
    every point up and to the right is dominated.
    """
    
    # Compute pareto set from full solution set.
    objective1_pareto, objective2_pareto = compute_pareto_set(objective1_list, 
                                                              objective2_list)
    
    # Sort by first objective.
    objective1_pareto, objective2_pareto = list(zip(*sorted(zip(objective1_pareto,
                                                                objective2_pareto))))
    
    # Compute the coordinates to plot.
    xs = []
    ys = []
    
    xs.append(objective1_pareto[0])
    ys.append(objective2_pareto[0])
    
    for i in range(0, len(objective1_pareto)-1):
        
        # Add intermediate point between successive solutions
        xs.append(objective1_pareto[i+1])
        ys.append(objective2_pareto[i])
        
        # Add next solution on front
        xs.append(objective1_pareto[i+1])
        ys.append(objective2_pareto[i+1])
        
    plt.plot(xs, ys)

In [ ]:
sns.set_theme()

# Plot overall stringency and cases of each prescription

for test_name in df['TestName'].unique():  # plot for each test
    plt.figure(figsize=(10,8))
    for prescriptor_name in prescription_files:
        pdf = df[df['PrescriptorName'] == prescriptor_name]
        pdf = pdf[pdf['TestName'] == test_name]
        overall_pdf = pdf.groupby(['PrescriptionIndex', ]).mean().reset_index()
        plt.scatter(overall_pdf['Stringency'],
                    overall_pdf['PredictedDailyNewCases'], 
                    label=prescriptor_name)
        plot_pareto_curve(list(overall_pdf['Stringency']),
                         list(overall_pdf['PredictedDailyNewCases']))
    plt.xlabel('Mean stringency')
    plt.ylabel('Mean cases per day per geo')
    plt.title('All Regions' + '-' + test_name)
    plt.legend()
    plt.show()

In [ ]:
# Plot stringency and cases of each prescription for a particular country and specific test

country_name = 'Canada'
test_name = 'Jan_Mar_RC_fast'

cdf = df[df.CountryName == country_name]
cdf = cdf[cdf['TestName'] == test_name]

plt.figure(figsize=(10,8))
for prescriptor_name in prescription_files:
    pdf = cdf[cdf['PrescriptorName'] == prescriptor_name]
    plt.scatter(pdf['Stringency'],
                pdf['PredictedDailyNewCases'], 
                label=prescriptor_name)
    plot_pareto_curve(list(pdf['Stringency']),
                      list(pdf['PredictedDailyNewCases']))
plt.xlabel('Mean stringency')
plt.ylabel('Mean cases per day')
plt.title(country_name + '-' + test_name)
plt.legend()
plt.show()
